In [14]:
from PIL import Image, ImageEnhance, ImageFilter, ImageOps
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\\Program Files\\Tesseract-OCR\\tesseract.exe"

We are going to use pyteseract to extract min and max temps so we have the scale range.

In [15]:
def run_ocr_on_coords(image_path):
    # Hardcoded coordinates for min/max temp regions
    coords = {
        "min_top_left": {"x": 36, "y": 31, "width": 34, "height": 15},
        "max_top_left": {"x": 35, "y": 45, "width": 35, "height": 17},
        "max_right": {"x": 197, "y": 38, "width": 38, "height": 18},
        "min_right": {"x": 199, "y": 261, "width": 36, "height": 19}
    }

        # Load the image
    img = Image.open(image_path)

    # This dictionary will store for each region both the OCR text and its confidence
    results = {}

    for label, coord in coords.items():
        # Extract coordinates
        x, y, w, h = coord["x"], coord["y"], coord["width"], coord["height"]
        cropped = img.crop((x, y, x + w, y + h))
        
        # Preprocess: grayscale, contrast enhancement, and sharpening
        gray = ImageOps.grayscale(cropped)
        enhanced = ImageEnhance.Contrast(gray).enhance(2.0)
        sharp = enhanced.filter(ImageFilter.SHARPEN)
        
        # Use image_to_data to get detailed OCR results including confidence values
        data = pytesseract.image_to_data(
            sharp,
            output_type=pytesseract.Output.DICT,
            config='--psm 6 -c tessedit_char_whitelist=0123456789.'
        )
        
        best_text = ""
        best_conf = -1  # Initialize with -1 to handle non-detections
        # Iterate through detected words (or fragments)
        for i in range(len(data['text'])):
            text_candidate = data['text'][i].strip()
            try:
                # Confidence returned as string; convert it to a float
                conf = float(data['conf'][i])
            except ValueError:
                conf = -1  # or simply skip if conversion fails
            # Pick the candidate with the highest confidence
            if text_candidate and conf > best_conf:
                best_text = text_candidate
                best_conf = conf

        results[label] = {"value": best_text, "conf": best_conf}

    # For regions where the same value is expected (duplicates), select the one with higher confidence.
    def choose_better(region1, region2):
        r1 = results.get(region1)
        r2 = results.get(region2)
        if r1 and r2:
            return r1 if r1["conf"] >= r2["conf"] else r2
        return r1 or r2

    final_min = choose_better("min_top_left", "min_right")
    final_max = choose_better("max_top_left", "max_right")

    # Return final chosen values and associated confidence scores.
    return {
        "min": final_min["value"], "min_conf": final_min["conf"],
        "max": final_max["value"], "max_conf": final_max["conf"]
    }




# Example usage
if __name__ == "__main__":
    image_path = "thermal_imges_for_exp\CH01.jpeg"  # Replace with your path
    results = run_ocr_on_coords(image_path)
    for key, val in results.items():
        print(f"{key}: {val}")


min: 49.9
min_conf: 70.0
max: 71.8
max_conf: 85.0


In [18]:
import numpy as np
from PIL import Image

def iterative_smoothing_newcolor(image_path, output_path, white_tolerance=100, similarity_tolerance=50, iterations=5):
    """
    Iteratively smooth near-white pixels by averaging over similar neighbors in a 5x5 window.
    
    For each pixel that is near white (each channel ≥ 255 - white_tolerance), the script:
      - Extracts its 5x5 neighborhood.
      - Selects pixels whose color values are within a similarity_tolerance (per channel)
        of the center pixel’s current value.
      - Averages these similar pixels to update the center pixel’s color.
    
    The process is repeated for a number of iterations. The update is done only on near-white pixels.
    
    Parameters:
        image_path (str): Path to the input image.
        output_path (str): Path to save the smoothed image.
        white_tolerance (int): A pixel is considered near white if each channel is ≥ 255 - white_tolerance.
        similarity_tolerance (int): A neighbor is “similar” if the absolute difference in each channel 
                                    is ≤ similarity_tolerance.
        iterations (int): Number of times to run the smoothing pass.
    
    Returns:
        int: The total number of pixel updates performed.
    """
    # Load image and convert to an RGB numpy array.
    img = Image.open(image_path).convert("RGB")
    img_np = np.array(img)
    
    height, width, channels = img_np.shape
    total_updates = 0
    current_img = img_np.copy()
    
    # A helper function to check if a pixel is near white.
    def is_near_white(pixel):
        return np.all(pixel >= (255 - white_tolerance))
    
    for it in range(iterations):
        updates_in_iteration = 0
        updated_img = current_img.copy()
        
        # Identify coordinates of near-white pixels.
        near_white_coords = np.argwhere(np.all(current_img >= (255 - white_tolerance), axis=-1))
        
        for i, j in near_white_coords:
            center_pixel = current_img[i, j]
            
            # Define the boundaries of the 5x5 window (with proper edge handling)
            i_min = max(i - 2, 0)
            i_max = min(i + 3, height)
            j_min = max(j - 2, 0)
            j_max = min(j + 3, width)
            
            # Extract the neighborhood window.
            window = current_img[i_min:i_max, j_min:j_max]
            
            # Determine which pixels in the window have colors similar to the center pixel.
            # The condition is that the absolute difference in each channel is within similarity_tolerance.
            diff = np.abs(window - center_pixel)  # Difference for each pixel (shape: [window_height, window_width, channels])
            similar_mask = np.all(diff <= similarity_tolerance, axis=-1)
            
            # Use only those pixels that are deemed similar.
            similar_pixels = window[similar_mask]
            
            if similar_pixels.size > 0:
                new_color = similar_pixels.mean(axis=0).astype(np.uint8)
                if not np.array_equal(center_pixel, new_color):
                    updated_img[i, j] = new_color
                    updates_in_iteration += 1

        current_img = updated_img
        total_updates += updates_in_iteration
        print(f"Iteration {it+1}: {updates_in_iteration} pixels updated.")
        
        # Optionally, if no pixel was updated during an iteration, we can break early.
        if updates_in_iteration == 0:
            break

    # Save the final image.
    smoothed_image = Image.fromarray(current_img)
    smoothed_image.save(output_path)
    print(f"Final smoothed image saved to: {output_path}")
    print(f"Total pixel updates: {total_updates}")
    return total_updates

if __name__ == "__main__":
    # Specify the path to your input image and the output destination.
    image_path = "thermal_imges_for_exp\CH01.jpeg"          # Change to your image file path if needed.
    output_path = "thermal_imges_for_exp\CH01_smoothed.jpeg"  # The processed image will be saved here.
    
    iterative_smoothing_newcolor(image_path, output_path, white_tolerance=100, similarity_tolerance=10, iterations=5)


Iteration 1: 1444 pixels updated.
Iteration 2: 1437 pixels updated.
Iteration 3: 1334 pixels updated.
Iteration 4: 1161 pixels updated.
Iteration 5: 990 pixels updated.
Final smoothed image saved to: thermal_imges_for_exp\CH01_smoothed.jpeg
Total pixel updates: 6366
